# Team 13 - Big Data Project

Dataset used: http://jmcauley.ucsd.edu/data/amazon/
Format: csv
Info: This dataset has book ratings from the Amazon Marketplace. The columns included are user, item, rating, timestamp (in epoch).

We uploaded the books dataset into the Jupyter Notebook. The dataset did not include the column headers, so we renamed the columns to the appropriate headers. We have shown with the first 5 rows that the dataset has been imported correctly.

In [4]:
import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'mI-MbZ6AEmHfS83ADHbH1e-u4fjrHqRymc_AM61ZhD-C',
    'service_id': 'iam-ServiceId-de7629bb-45be-4b02-9da7-963fa26c02ba',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_c4f7ec2404e542d7975c82953f07934a_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'false')\
  .load(cos.url('ratings_Books.csv', 'project13bigdataca7d76b8c08445c79249a515ffeb56cb'))

In [5]:
df = df.withColumnRenamed("_c0","User")
df = df.withColumnRenamed("_c1","ProductId")
df = df.withColumnRenamed("_c2","Rating")
df = df.withColumnRenamed("_c3","epoch")
df.take(5)

[Row(User='AH2L9G3DQHHAJ', ProductId='0000000116', Rating='4.0', epoch='1019865600'),
 Row(User='A2IIIDRK3PRRZY', ProductId='0000000116', Rating='1.0', epoch='1395619200'),
 Row(User='A1TADCM7YWPQ8M', ProductId='0000000868', Rating='4.0', epoch='1031702400'),
 Row(User='AWGH7V0BDOJKB', ProductId='0000013714', Rating='4.0', epoch='1383177600'),
 Row(User='A3UTQPQPM4TQO0', ProductId='0000013714', Rating='5.0', epoch='1374883200')]

In [40]:
df.printSchema()

root
 |-- User: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- epoch: string (nullable = true)
 |-- timestamp: string (nullable = true)



## Timestamp Conversion from epoch to string

In [7]:
from pyspark.sql.functions import *
df = df.withColumn("timestamp",from_unixtime(col("epoch")))

In [38]:
df.printSchema()

root
 |-- User: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- epoch: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [9]:
df.take(5)

[Row(User='AH2L9G3DQHHAJ', ProductId='0000000116', Rating='4.0', epoch='1019865600', timestamp='2002-04-26 19:00:00'),
 Row(User='A2IIIDRK3PRRZY', ProductId='0000000116', Rating='1.0', epoch='1395619200', timestamp='2014-03-23 19:00:00'),
 Row(User='A1TADCM7YWPQ8M', ProductId='0000000868', Rating='4.0', epoch='1031702400', timestamp='2002-09-10 19:00:00'),
 Row(User='AWGH7V0BDOJKB', ProductId='0000013714', Rating='4.0', epoch='1383177600', timestamp='2013-10-30 19:00:00'),
 Row(User='A3UTQPQPM4TQO0', ProductId='0000013714', Rating='5.0', epoch='1374883200', timestamp='2013-07-26 19:00:00')]

# Data Analysis

We grouped the data on Rating and calculated the count to analyze how many were of each rating. From the table below, we discovered that users gave an overwhelming majority of 5.0 ratings over any other rating. Additionally ratings leaned more toward higher ratings; however the number of people who gave the worst rating is more than the number who gave a 2.0 rating.

In [22]:
gr = df.groupBy("Rating").count().sort("Rating")

In [2]:
# Build the SQL context required to create a Spark dataframe 
sqlContext=SQLContext(sc) 

## Products with most ratings

In the below query, we analyze the ten products that were given the largest number of ratings, sorted in descending order.

In [41]:
df.groupBy("ProductId").count().sort(col("count").desc()).show(10)

+----------+-----+
| ProductId|count|
+----------+-----+
|0439023483|21398|
|030758836X|19867|
|0439023513|14114|
|0385537859|12973|
|0007444117|12629|
|0375831002|12571|
|038536315X|12564|
|0345803485|12290|
|0316055433|11746|
|0849922070|10424|
+----------+-----+
only showing top 10 rows



In [42]:
df_2 = df.withColumn("year", col("timestamp").substr(0,4))

## Ratings per year

In [45]:
df_2.groupBy("year").count().sort(col("year").desc()).show()

+----+-------+
|year|  count|
+----+-------+
|2014|4728411|
|2013|6994009|
|2012|2931573|
|2011|1525596|
|2010|1131037|
|2009| 977937|
|2008| 783409|
|2007| 713494|
|2006| 543011|
|2005| 479083|
|2004| 337309|
|2003| 288616|
|2002| 284875|
|2001| 295022|
|2000| 322493|
|1999| 103225|
|1998|  55800|
|1997|  12218|
|1996|     37|
+----+-------+



## Rating counts per year ordered by count

In [47]:
df_2.groupBy("year").count().sort(col("count").desc()).show()

+----+-------+
|year|  count|
+----+-------+
|2013|6994009|
|2014|4728411|
|2012|2931573|
|2011|1525596|
|2010|1131037|
|2009| 977937|
|2008| 783409|
|2007| 713494|
|2006| 543011|
|2005| 479083|
|2004| 337309|
|2000| 322493|
|2001| 295022|
|2003| 288616|
|2002| 284875|
|1999| 103225|
|1998|  55800|
|1997|  12218|
|1996|     37|
+----+-------+



In [51]:
counts = df_2.groupBy(['year', 'User']).count().alias('counts')
counts.show(20)

+----+--------------+-----+
|year|          User|count|
+----+--------------+-----+
|2006| AFYCJ0BY9XX84|    1|
|2014|A1BWIXPFCVF2RM|    1|
|2007| ADE36E3UI2T08|    6|
|2007|A2B63ED0F870AT|    1|
|2013| AN5Q5TGABA5WY|    3|
|2008|A1EIFIMA43D8UN|    1|
|2014|A2CD9BZSYUJZ1Q|    4|
|2014|A3TRELWZQHPSVT|    3|
|2013|A2LDDHKECKIM1Y|   12|
|2008|A2MVUWT453QH61|   49|
|2010|A3CQ5X4DM1GCK7|    8|
|2010|A25FW4MUV5206K|    2|
|2010|A269FUWRKV5M7N|    2|
|2009|A27KA2H3E5C14S|    1|
|2014|A1QVWXDUMVDMJ2|    2|
|2011|A2NX1VF0HK4BSR|    6|
|2013| A8I6ZL027SCFZ|    2|
|2013|A3LI3BS69XXNBG|    1|
|2000|  ANKOIBS1MV5H|    3|
|2000| ACELKH5KHQ752|    1|
+----+--------------+-----+
only showing top 20 rows



## Rating providers per year

In [83]:
counts.groupBy('year').count().sort(col("year").desc()).show()

+----+-------+
|year|  count|
+----+-------+
|2014|2224276|
|2013|3000014|
|2012|1495209|
|2011| 815658|
|2010| 625652|
|2009| 539602|
|2008| 430136|
|2007| 394424|
|2006| 298852|
|2005| 262755|
|2004| 179255|
|2003| 152942|
|2002| 148924|
|2001| 157058|
|2000| 184189|
|1999|  74670|
|1998|  39499|
|1997|   9053|
|1996|     13|
+----+-------+

